In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from keras import backend as K
from keras.layers import Layer,InputSpec
import keras.layers as kl
from glob import glob
from sklearn.metrics import roc_curve, auc
from keras.preprocessing import image
from tensorflow.keras.models import Sequential
from sklearn.metrics import roc_auc_score
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from  matplotlib import pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.layers import concatenate,Dense, Conv2D, MaxPooling2D, Flatten,Input,Activation,add,AveragePooling2D,BatchNormalization,Dropout
%matplotlib inline
import shutil
from sklearn.metrics import  precision_score, recall_score, accuracy_score,classification_report ,confusion_matrix
from tensorflow.python.platform import build_info as tf_build_info
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


In [18]:
# gettig data
data = pd.read_csv('/content/drive/MyDrive/Project_Brain tumor/Dataset/final_metadata.csv')
data.head()

,ID,Label,ID
0,NaN,1.0,1.0
1,NaN,1.0,2.0
2,NaN,1.0,3.0
3,NaN,1.0,4.0
4,NaN,1.0,5.0


In [19]:


train_dir = "/content/drive/MyDrive/Project_Brain tumor/Data/Train"
test_dir  =  "/content/drive/MyDrive/Project_Brain tumor/Data/Test"


os.mkdir(train_dir)
os.mkdir(test_dir)

In [20]:
train_df , test_df = train_test_split(data, test_size=0.15, stratify=data['Label'])

In [21]:
train_df.to_csv("/content/drive/MyDrive/Project_Brain tumor/Data/train")

In [22]:
test_df.head()

,ID,Label,ID
1772,NaN,3.0,1773.0
2868,NaN,2.0,2869.0
1642,NaN,3.0,1643.0
2891,NaN,2.0,2892.0
73,NaN,1.0,74.0


In [23]:
# Image id of train and test images
train_list = list(train_df['ID'])
test_list = list(test_df['ID'])

In [24]:
# 1 for meningioma, 2 for glioma, 3 for pituitary tumor
# and also we want to change the path of the data and split them into their calsses
target = ['1' , '2' , '3']
for i in target:
  train_file = train_dir+'/'+i
  test_file = test_dir+'/'+i
  os.mkdir(train_file)
  os.mkdir(test_file)

In [25]:
for item in train_list:
  # this os the sourc of all of the images
   source  = '/content/drive/MyDrive/Project_Brain tumor/Dataset/final_data/'+str(int(item))+'.jpg'
  # we have to copy the images from source to the directory which related to its label
   Picture_ID = train_df.loc[train_df["ID"] == item , 'Label'].iloc[0]

   target = '/content/drive/MyDrive/Project_Brain tumor/Data/Train/'+str(int(Picture_ID))+'/'+str(int(item))+'.jpg'

   shutil.copyfile(source, target)


In [26]:
for item in test_list:
  # this os the sourc of all of the images
   source  = '/content/drive/MyDrive/Project_Brain tumor/Dataset/final_data/'+str(int(item))+'.jpg'
  # we ha1.0ve to copy the images from source to the directory which related to its label
   Picture_ID = test_df.loc[test_df["ID"] == item , 'Label'].iloc[0]

   target = '/content/drive/MyDrive/Project_Brain tumor/Data/Test/'+str(int(Picture_ID))+'/'+str(int(item))+'.jpg'

   shutil.copyfile(source, target)


In [27]:
# now we have to use augmentation


In [28]:
train_df.head()

,ID,Label,ID
170,NaN,1.0,171.0
2599,NaN,2.0,2600.0
1175,NaN,3.0,1176.0
791,NaN,2.0,792.0
126,NaN,1.0,127.0


In [29]:
#  in the code below we try to generate some augmented pictures
target = ['1'  , '2' , '3']

for imag_calss in target :

  os.mkdir("/content/contemperory_images1")
  os.mkdir("/content/contemperory_images1/dir")

  image_list = os.listdir("/content/drive/MyDrive/Project_Brain tumor/Data/Train/"+ imag_calss)

  for image_file in image_list:

        source = os.path.join('/content/drive/MyDrive/Project_Brain tumor/Data/Train/' + imag_calss, image_file)

        target = os.path.join("/content/contemperory_images1/dir" , image_file)

        shutil.copyfile(source, target)

  save_aug = '/content/drive/MyDrive/Project_Brain tumor/Data/Train/' + imag_calss
  datagen = tf.keras.preprocessing.image.ImageDataGenerator(

        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'

    )
  batch_size = 50

  aug_datagen = datagen.flow_from_directory("/content/contemperory_images1",save_to_dir=save_aug,save_format='jpg',target_size=(224, 224),batch_size=batch_size)

    # Generate the augmented images
  aug_images = 8000

  num_files = len(os.listdir("/content/contemperory_images1/dir"))
  print(num_files)
  num_batches = int(np.ceil((aug_images - num_files) / batch_size))

    # creating 8000 augmented images per class
  for i in range(0, num_batches):
        images, labels = next(aug_datagen)

    # delete temporary directory
  shutil.rmtree("/content/contemperory_images1")



Found 601 images belonging to 1 classes.
601
Found 1212 images belonging to 1 classes.
1212
Found 790 images belonging to 1 classes.
790
